In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [2]:
#importando datos de fallas
dff=df = pd.read_excel('EPM_datos_Uraba\Original_Data\Reporte Uraba2019_CAGMV1Est.xlsx', sheet_name='BD-Filtrada')
dff["town"]=dff["town"].str.upper()

In [3]:
#importando datos de clima

precip_path="EPM_datos_Uraba/weather data/precipitacion/"
dfw_precip = pd.DataFrame()
for file_name in glob.glob(precip_path+'*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    dfw_precip = pd.concat([dfw_precip,x],axis=0)
    
hum_path="EPM_datos_Uraba/weather data/hum/"
dfw_hum = pd.DataFrame()
for file_name in glob.glob(hum_path+'*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    dfw_hum = pd.concat([dfw_precip,x],axis=0)
    
#hum_path="EPM_datos_Uraba\weather data\hum\""
#dfw_precip = pd.DataFrame()
#for file_name in glob.glob(hum_path+'*.csv'):
#    x = pd.read_csv(file_name, low_memory=False)
#    glued_data = pd.concat([glued_data,x],axis=0)
dfw_precip["Municipio"]=dfw_precip["Municipio"].str.upper()
dfw_hum["Municipio"]=dfw_hum["Municipio"].str.upper()

In [4]:
#importando los datos de infraestructura

#transformadores

dfi_t=df = pd.read_csv('EPM_datos_Uraba\Original_Data\TRANSFO_PT.csv')

In [5]:
#importando divipola

divipola= pd.read_excel('EPM_datos_Uraba\municipios\Listados_DIVIPOLA.xlsx', sheet_name='municipio')
divipola= divipola[divipola["Nombre_depto"].isin(["ANTIOQUIA","CHOCÓ"])]
divipola["Nombre_depto"].unique()

array(['ANTIOQUIA', 'CHOCÓ'], dtype=object)

In [6]:
#itenditificando las variables que identifican el municipio
#dff.town #nombre minúscula
#dfw_precip.Municipio #nombre
#dfi_t.MUNICIPIO_ #código 4 dígitos
#divipola.Nombre_municipio # nombre mayúscula
#divipola.Código_municipio # código 4 dígitos

In [7]:
#incluyendo el nombre del municipio en la base de insfraestructura

dfi_tm=dfi_t.merge(divipola[["Código_municipio","Nombre_municipio", "Nombre_depto", "Código_depto "]], left_on="MUNICIPIO_", right_on="Código_municipio")

In [8]:
#incluyendo el código del municipio en la base de fallas
dff_m=dff.merge(divipola[["Código_municipio","Nombre_municipio", "Nombre_depto", "Código_depto "]], left_on="town", right_on="Nombre_municipio")



In [9]:
#incluyendo el código del municipio en las bases de precipitacion y humedad

dfw_precip=dfw_precip.merge(divipola[["Código_municipio","Nombre_municipio", "Nombre_depto", "Código_depto "]], left_on="Municipio", right_on="Nombre_municipio")
dfw_hum=dfw_hum.merge(divipola[["Código_municipio","Nombre_municipio", "Nombre_depto", "Código_depto "]], left_on="Municipio", right_on="Nombre_municipio")

In [10]:
print(len(dfi_tm))
print(len(dff_m))
print(len(dff))
#hay algunos datos qde fallas que no podemos cruzar porque no está el municipio

13503
44271
49494


In [11]:
dff_m.Nombre_depto.unique()

array(['ANTIOQUIA', 'CHOCÓ'], dtype=object)

In [12]:
dfi_tm.Nombre_depto.unique()

array(['ANTIOQUIA', 'CHOCÓ'], dtype=object)

In [13]:
dfw_precip.Nombre_depto.unique()

array(['ANTIOQUIA', 'CHOCÓ'], dtype=object)

In [14]:
dfw_hum.Nombre_depto.unique()

array(['ANTIOQUIA', 'CHOCÓ'], dtype=object)

In [15]:
#adding a dummy variable indicating if there was a failure
dff_m["fail"]=1

In [82]:
#fails_circuit_by_day_loca=pd.DataFrame(pd.unique(dff_m[["NumberOT","Circuit","OpenDate","fail"]].values.ravel()))
#fails_circuit_by_day_loca.head()

distinct_df = dff_m[ ["NumberOT","Circuit","OpenDate","Nombre_municipio","fail"] ].groupby(["NumberOT","Circuit","OpenDate","Nombre_municipio","fail"]).first().reset_index()
distinct_df.head()


,NumberOT,Circuit,OpenDate,Nombre_municipio,fail
0,2831866,412-14,2018-08-30 17:32:28,CAREPA,1
1,2929866,406-11,2018-12-07 09:48:58,TURBO,1
2,2934398,415-11,2018-12-13 07:37:00,NECOCLÍ,1
3,2945255,407-12,2018-12-28 09:27:36,SAN PEDRO DE URABÁ,1
4,2945565,415-13,2018-12-28 16:30:01,NECOCLÍ,1


In [83]:
print(distinct_df.shape)
print(dff_m.shape)

(30644, 5)
(44271, 41)


In [68]:
#creating false failures to complement no failures

#list of unique cicuits
#circuits_f=pd.DataFrame(pd.unique(dff_m["Circuit"]))
#print(circuits_f.shape)


circuits_i=pd.DataFrame(pd.unique(dfi_tm["CIRCUITO"]))
print(circuits_i.shape)


(70, 1)


In [69]:
#Probando regresión Poisson
from patsy import dmatrices
import statsmodels.api as sm

In [99]:
pd.to_datetime(distinct_df['OpenDate'], format='%Y-%m-%d %H %M %S')
distinct_df['Date']=distinct_df['OpenDate'].dt.date
distinct_df['day']=distinct_df['OpenDate'].dt.day
distinct_df['Month']=distinct_df['OpenDate'].dt.month
distinct_df.columns


Index(['NumberOT', 'Circuit', 'OpenDate', 'Nombre_municipio', 'fail', 'Date',
       'day', 'Month'],
      dtype='object')

In [100]:
distinct_df1=distinct_df[["day","Month","Nombre_municipio","Circuit","NumberOT"]].groupby(["day","Month","Nombre_municipio","Circuit"]).count().reset_index()
distinct_df1.head()

,day,Month,Nombre_municipio,Circuit,NumberOT
0,1,1,ARBOLETES,409-11,1
1,1,1,ARBOLETES,409-12,1
2,1,1,CAREPA,412-12,1
3,1,1,CAREPA,412-13,1
4,1,1,CAREPA,412-14,3


In [104]:
mask = np.random.rand(len(distinct_df1)) < 0.8
df_train = distinct_df1[mask]
df_test = distinct_df1[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))

Training data set length=9547
Testing data set length=2467


In [107]:
#espcifiación del modelo

expr = """NumberOT ~ day + Month  + Nombre_municipio + Circuit """

y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')


poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:               NumberOT   No. Observations:                 9547
Model:                            GLM   Df Residuals:                     9458
Model Family:                 Poisson   Df Model:                           88
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -18488.
Date:                Fri, 10 Jul 2020   Deviance:                       12264.
Time:                        18:27:32   Pearson chi2:                 1.81e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [103]:
poisson_predictions = poisson_training_results.get_prediction(X_test)
#.summary_frame() returns a pandas DataFrame
predictions_summary_frame = poisson_predictions.summary_frame()
print(predictions_summary_frame)
############revisar los datos para eliminar los NaN y eliminarlos.

ValueError: shapes (2430,76) and (90,) not aligned: 76 (dim 1) != 90 (dim 0)